In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# BCE classifier

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
from pytorch_inferno.model_wrapper import ModelWrapper
from pytorch_inferno.callback import *
from pytorch_inferno.data import get_paper_data
from pytorch_inferno.plotting import *
from pytorch_inferno.inference import *
from pytorch_inferno.utils import *

from fastcore.all import partialler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch.nn as nn
from torch import optim
import torch
from torch import Tensor

In [ ]:
net = nn.Sequential(nn.Linear(3,100),  nn.ReLU(),
                    nn.Linear(100,100),nn.ReLU(),
                    nn.Linear(100,1),  nn.Sigmoid())
init_net(net)

In [ ]:
model = ModelWrapper(net)

data, test = get_paper_data(200000, bm=0, bs=64, n_test=1000000)

In [ ]:
model.fit(200, data=data, opt=partialler(optim.SGD,lr=2e-3), loss=nn.BCELoss(),
          cbs=[LossTracker(),EarlyStopping(5)])

Train: 0.47231031057357786 Valid: 0.4106015536403656
Train: 0.3764215631723404 Valid: 0.35775666311264037
Train: 0.34840640859603883 Valid: 0.34506445516586304
Train: 0.3413417801809311 Valid: 0.3438991569519043
Train: 0.3393236142730713 Valid: 0.33977342045784
Train: 0.33844144454956054 Valid: 0.3393754963874817
Train: 0.33780056032657624 Valid: 0.33978004629135133
Train: 0.3369659703683853 Valid: 0.3384995010280609
Train: 0.33632265988349913 Valid: 0.3377543615055084
Train: 0.3359568090677261 Valid: 0.3375732827568054
Train: 0.33574455837249756 Valid: 0.33751689333915713
Train: 0.3355265342378616 Valid: 0.3393559790897369
Train: 0.33520790438652037 Valid: 0.3370229545593262
Train: 0.33503071492671965 Valid: 0.33649231322288514
Train: 0.334813419675827 Valid: 0.33676126938819884
Train: 0.3345243829345703 Valid: 0.3363676014328003
Train: 0.33434586748838424 Valid: 0.3364793776321411
Train: 0.3339301758813858 Valid: 0.3362315066623688
Train: 0.3338347667264938 Valid: 0.3362987785053253


In [ ]:
preds = model.predict(test)

In [ ]:
preds

In [ ]:
df = pd.DataFrame({'pred':preds.squeeze()})
df['gen_target'] = test.dataset.y
df.head()

In [ ]:
plot_preds(df)

In [ ]:
bin_preds(df)

In [ ]:
df.head()

In [ ]:
df['gen_weight'] = 1
df.loc[df.gen_target == 0, 'gen_weight'] = 1000/len(df[df.gen_target == 0])
df.loc[df.gen_target == 1, 'gen_weight'] = 50/len(df[df.gen_target == 1])

In [ ]:
df.head()

In [ ]:
df.gen_weight.sum()

In [ ]:
mu = 50/1000; mu

In [ ]:
f_s = df.loc[df.gen_target == 1, 'pred_bin'].value_counts()
f_s.sort_index(inplace=True)
f_s /= f_s.sum()
f_b = df.loc[df.gen_target == 0, 'pred_bin'].value_counts()
f_b.sort_index(inplace=True)
f_b /= f_b.sum()

In [ ]:
f_s.sum(), f_b.sum()

In [ ]:
f_s, f_b

In [ ]:
t = f_s/(f_s+f_b); t

In [ ]:
plt.plot(t)

In [ ]:
t_exp = ((50*f_s)+(1000*f_b)).values

In [ ]:
m = 50
n = 1050
((m*f_s)+((n-m)*f_b)).sum()

In [ ]:
t_exp = Tensor(t_exp)

In [ ]:
t_exp, t_exp.sum()

In [ ]:
m = 50
pois = torch.distributions.Poisson(((m*Tensor(f_s))+((n-m)*Tensor(f_b))))

In [ ]:
-torch.distributions.Poisson(t_exp).log_prob(t_exp).mean()

In [ ]:
-pois.log_prob(Tensor(t_exp)).mean()

In [ ]:
x = np.linspace(20,80,61)
y = np.zeros_like(x)
for i,m in enumerate(x):
    pois = torch.distributions.Poisson(((m*Tensor(f_s))+((n-m)*Tensor(f_b))))
    y[i] = -pois.log_prob(Tensor(t_exp)).mean()
y -= y.min()

In [ ]:
plt.plot(x,y)

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline

In [ ]:
r = InterpolatedUnivariateSpline(x, y-0.05).roots()

In [ ]:
(r[1]-r[0])/2